In [1]:
from data_loader import get_mix_instruct

prompts, references, ds_name = get_mix_instruct("train", 1000)

Dataset: mix-instruct_train_1000 found in cache, loading from cache ✅


In [2]:
from utility_functions.delift_se import get_delift_se_utility

utility, utility_name = get_delift_se_utility(prompts, references, ds_name)

Utility: mix-instruct_train_1000_delift-se found in cache, loading from cache ✅


In [3]:
from subset import create_subset

subset, subset_name = create_subset(utility, utility_name)

Subset: mix-instruct_train_1000_delift-se_0.3 found in cache, loading from cache ✅


In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

device = 'cuda:0'
model_name = "microsoft/Phi-3-mini-128k-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True, attn_implementation="flash_attention_2",torch_dtype=torch.bfloat16).to(device)

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
from torch.utils.data import DataLoader
from tqdm import tqdm

def generate_responses(prompts, tokenizer, model, batch_size=8, max_length=512):
    dataloader = DataLoader(prompts, batch_size=batch_size, shuffle=False)
    all_responses = []

    for batch in tqdm(dataloader, desc="Generating responses"):
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True).to(device)
        with torch.no_grad():
            outputs = model.generate(**inputs, max_new_tokens=max_length, do_sample=False)
        responses = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
        all_responses.extend(responses)

    return all_responses


generate_responses(prompts[:100], tokenizer, model)

Generating responses:   0%|          | 0/13 [00:00<?, ?it/s]


AttributeError: 'DynamicCache' object has no attribute 'get_max_length'

In [9]:
dataloader = DataLoader(prompts, batch_size=8, shuffle=False)
batch = next(iter(dataloader))
inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True).to(device)

outputs = model.generate(
    **inputs,
    max_new_tokens=512,
    do_sample=False,
)

AttributeError: 'DynamicCache' object has no attribute 'get_max_length'